<a href="https://colab.research.google.com/github/steveding1/DA-learning/blob/master/NY_weather_vs_electricty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import urllib.request
import re
from datetime import datetime
import time 
import pytz

In [ ]:
datetime.now(pytz.timezone("America/New_York"))

datetime.datetime(2022, 7, 19, 18, 29, 0, 222731, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)

'http://mis.nyiso.com/public/csv/pal/20220719pal.csv'

In [ ]:
weath_url = 'https://forecast.weather.gov/MapClick.php?textField1=40.78&textField2=-73.97#.YtWW3HaZNhE'
elect_url = 'http://mis.nyiso.com/public/csv/pal/{yyyymmdd}pal.csv'.format(time.strftime("%Y%m%d"))


In [ ]:
weath_df = pd.read_html(weath_url)[0].set_index(0).transpose()

In [ ]:
weath_df

,Humidity,Wind Speed,Barometer,Dewpoint,Visibility,Heat Index,Last update
1,41%,Vrbl 6 mph,29.78 in (1007.4 mb),63°F (17°C),10.00 mi,91°F (33°C),19 Jul 4:51 pm EDT


In [ ]:
weath_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 1 to 1
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Humidity     1 non-null      int64         
 1   Wind Speed   1 non-null      object        
 2   Barometer    1 non-null      int64         
 3   Dewpoint     1 non-null      int64         
 4   Visibility   1 non-null      int64         
 5   Heat Index   1 non-null      int64         
 6   Last update  1 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 64.0+ bytes


In [ ]:
weath_df['Humidity'] = weath_df['Humidity'].str.extract('(\d+)').astype(int)
weath_df['Barometer'] = weath_df['Barometer'].str.findall('(\d+)').str[1].astype(int)
weath_df['Dewpoint'] = weath_df['Dewpoint'].str.findall('(\d+)').str[1].astype(int)
weath_df['Visibility'] = weath_df['Visibility'].str.findall('(\d+)').str[1].astype(int)
weath_df['Heat Index'] = weath_df['Heat Index'].str.findall('(\d+)').str[1].astype(int)
weath_df['Last update'] = weath_df['Last update'].str.replace(' EDT','')+str(datetime.now().year)
weath_df['Last update'] = pd.to_datetime(weath_df['Last update'], format='%d %b %I:%M %p%Y')
weath_df

,Humidity,Wind Speed,Barometer,Dewpoint,Visibility,Heat Index,Last update
1,45,Vrbl 5 G 17 mph,77,18,0,33,2022-07-19 17:51:00


In [ ]:
from os.path import exists
weath_csv = 'weath_tmp.csv'
if exists('weath_tmp.csv'):
  pd.read_csv(weath_csv,parse_dates=[-1],infer_datetime_format='Y-m-D H:M:S').append(weath_df).drop_duplicates(subset=['Last update'], keep='last').to_csv(weath_csv,index=False)
else:
  weath_df.to_csv(weath_csv,index=False)

In [ ]:
pd.read_csv(weath_csv,parse_dates=[-1],infer_datetime_format='Y-m-D H:M:S')

,Humidity,Wind Speed,Barometer,Dewpoint,Visibility,Heat Index,Last update
0,41,Vrbl 6 mph,78,17,0,33,2022-07-19 16:51:00
1,45,Vrbl 5 G 17 mph,77,18,0,33,2022-07-19 17:51:00


In [ ]:
nydate = datetime.now(pytz.timezone("America/New_York")).strftime("%Y%m%d")
elect_url = 'http://mis.nyiso.com/public/csv/pal/{0}pal.csv'.format(nydate)
csvfname = 'electricity.csv'
elect_df = pd.read_csv(elect_url,parse_dates=[0],infer_datetime_format='Y-m-D H:M:S')
elect_df = elect_df[elect_df['Name']=='N.Y.C.'][['Time Stamp','Load']].dropna(subset=['Load'])
elect_df
if exists(csvfname):
    pd.read_csv(csvfname,header=0,parse_dates=[0],infer_datetime_format='Y-m-D H:M:S').append(elect_df).drop_duplicates(subset=['Time Stamp'], keep='last').to_csv(csvfname,index=False)
else:
    elect_df.to_csv(csvfname,index=False)

In [ ]:
elect_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305 entries, 8 to 3352
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Time Stamp  305 non-null    datetime64[ns]
 1   Load        305 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 7.1 KB


In [ ]:
pd.read_csv(csvfname,header=0).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Time Stamp  450 non-null    object 
 1   Load        450 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.2+ KB
